# Problem 1 – Binary Words and Operations  
**Author:** Michael Ferry  
**Date:** October 2025 

In [21]:
import numpy as np


## 32-bit Integers

In SHA-256 everything uses 32-bit numbers.  
Python doesn’t do that by default, so I made a small helper that turns any number into a 32-bit one.  
This makes sure the maths works the same way as in the standard.



I used the modulo (%) operator so the number always stays in the 32-bit range (0 to 4294967295).  
If it goes below 0 or above the limit, it wraps back around like a loop.  
This makes it act the same way as real 32-bit hardware does in SHA-256.

In [22]:
def to_uint32(x):

    return np.uint32(x % (1 << 32))

# Just runs a quick test to check the wrap
print(to_uint32(123))   # normal
print(to_uint32(-1))    # wraps around to 4294967295

123
4294967295


## Parity Function  

Parity checks three 32-bit numbers and returns 1 in places where an odd number of bits are 1.  
I used the XOR (^) operator because it flips bits this way and works perfectly for this rule.

In [23]:
def Parity(x, y, z):
    """Gives 1 for each bit where an odd number of x, y, z bits are 1."""
    x, y, z = map(to_uint32, (x, y, z))
    return x ^ y ^ z

# Just runs a quick test of the parity function
print(bin(Parity(0b1010, 0b0101, 0b0011)))


0b1100


## Choice Function

The Choice function checks three 32-bit numbers and returns bits from **y** or **z** depending on **x**.  
If a bit in **x** is 1, the corresponding bit from **y** is chosen.  
If it’s 0, the bit from **z** is chosen.  


In [24]:
def Ch(x, y, z):
    """Returns bits from y where x has 1s otherwise from z."""
    x, y, z = map(to_uint32, (x, y, z))
    return (x & y) ^ (~x & z)

# Runs a test of the choice function
print(bin(Ch(0b1010, 0b1100, 0b0110)))


0b1100


## Majority Function

The Majority function checks three 32-bit numbers and returns 1 in each bit position where two or more of the bits are 1.  

This means the result bit is 1 if at least two of x, y, z have 1 in that position.


In [25]:
def Maj(x, y, z):
    """Gives 1 for each bit where at least two of x, y, z bits are 1."""
    x, y, z = map(to_uint32, (x, y, z))
    return (x & y) ^ (x & z) ^ (y & z)

# Just runs a quick test of the majority function
print(bin(Maj(0b1010, 0b1100, 0b0110)))

0b1110


### Σ₀ Function — $\Sigma_0^{\{256\}}(x)$

The Sigma 0 function uses three separate right rotations to a 32-bit input value and then combines them using XOR.
It’s the first of the two big Sigma operations used during the SHA-256 compression step.

By rotating the bits by different amounts and mixing them together, the function helps spread bit patterns more evenly across the word.
This improves diffusion, meaning that even small changes in the input will cause large changes in the resulting hash output when its run.


In [26]:
def ROTR(x, n):
    """Right rotates a 32-bit integer x by n bits."""
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF

def Sigma0(x):
    """Implements Σ₀^{256}(x) using rotations by 2, 13, and 22 bits."""
    return ROTR(x, 2) ^ ROTR(x, 13) ^ ROTR(x, 22)

# Quick test
print(bin(Sigma0(0x12345678)))


0b1100110000101000110010001110100


### Σ₁ Function — Sigma 1

The Sigma 1 function performs three separate right rotations on a 32-bit input value and then combines the results using XOR.  
It is the second of the two big Sigma functions used in the SHA-256 compression step.  

By rotating the bits by different amounts and mixing them, Sigma 1 helps increase diffusion, each bit of the input influences many bits in the output.  
This makes even the smallest change in the input produce a completely different result, which is essential for the hash function’s unpredictability.


In [ ]:
def Sigma1(x):
    """Uses Σ₁^{256}(x) using rotations by 6, 11, and 25 bits."""
    return ROTR(x, 6) ^ ROTR(x, 11) ^ ROTR(x, 25)

# Quick test
print(bin(Sigma1(0x12345678)))


0b110101011000011010101111011010
